# RegNet Inference (submission generation)

In [27]:
import os
import sys
from pathlib import Path
import joblib
from joblib import Parallel, delayed
from tqdm import tqdm
import glob
import pandas as pd
import torch
import torch.nn as nn
import torchaudio
import numpy as np
from torchvision.models import regnet_y_800mf, RegNet_Y_800MF_Weights
import timm
import re
from torchaudio import functional as F_audio
import pywt

In [28]:
## REUSE IN INFERENCE NOTEBOOK

custom_dataset_path = '/kaggle/input/birdclef2023-inference'
if os.path.exists(os.path.join(custom_dataset_path, 'utils.py')):
    sys.path.append(custom_dataset_path)
else:
    sys.path.append('..')
import utils

IS_IN_KAGGLE_ENV = utils.get_is_in_kaggle_env()

DATA_PATH = '/kaggle/input/birdclef-2023' if IS_IN_KAGGLE_ENV else '../data'
JOBLIB_PATH = custom_dataset_path if IS_IN_KAGGLE_ENV else './'

DEVICE = 'cpu'

AUDIO_LENGTH_S = 5
SAMPLE_RATE = 32_000

We are running code on Localhost


In [29]:
## REUSE IN INFERENCE NOTEBOOK

class BirdMelspecClf(nn.Module):
    def __init__(self, out_features, pretrained):
        super().__init__()
        
        # https://pytorch.org/vision/stable/models.html

        self.regnet = regnet_y_800mf(weights=RegNet_Y_800MF_Weights.DEFAULT) if pretrained else regnet_y_800mf()

        """
        Original:
        RegnetCNN(
        (regnet): RegNet(
            (stem): SimpleStemIN(
            (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU(inplace=True)
        )"""
        self.regnet.stem = nn.Sequential(
            nn.Conv2d(4, 32, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.ReLU(inplace=True),
        )
        
        # Fine-tune the regnet classifier
        self.regnet.fc = nn.Linear(self.regnet.fc.in_features, out_features)

        self.softmax = nn.Softmax(dim=1)
 
    def forward(self, x):
        logits = self.regnet(x)
        probas = self.softmax(logits)

        return logits, probas


def get_model(out_features, device, pretrained=False, load_state_dict=True, state_dict_starts_with=f"{AUDIO_LENGTH_S}s_regnetY800MF_"):
    model = BirdMelspecClf(out_features=out_features, pretrained=pretrained)
    print(f"Loaded model {model.__class__.__name__} with {sum(p.numel() for p in model.parameters())} parameters, pretained={pretrained}")
    model.to(device)

    if not load_state_dict:
        return model

    model_files = [f for f in os.listdir(JOBLIB_PATH) if f.startswith(state_dict_starts_with) and f.endswith('.pt')]
    if len(model_files) == 0:
        print(f"No model starting with {state_dict_starts_with} found in {JOBLIB_PATH}")
        return model
    
    # Extract timestamp from the filenames and sort based on it
    model_files.sort(key=lambda x: int(re.findall(r'\d+', x)[-1]) if re.findall(r'\d+', x) else -1)

    # The latest model file is the last one in the sorted list
    latest_model_file = model_files[-1]
    model_path = os.path.join(JOBLIB_PATH, latest_model_file)
    model.load_state_dict(torch.load(model_path))
    print(f"Loaded model weights from {model_path}")
    model.to(device)

    return model


def get_label_encoder():
    label_encoder_path = os.path.join(JOBLIB_PATH, 'label_encoder.joblib')
    label_encoder = joblib.load(label_encoder_path)
    print(f"Loaded label encoder from {label_encoder_path}")
    return label_encoder

In [30]:
class WaveletTransformSingle(nn.Module):
  def __init__(
      self, 
      wavelet: pywt.Wavelet,
      cut_to_nearest: int | None = None
  ):
    super(WaveletTransformSingle, self).__init__()
    self.wavelet = wavelet
    self.ctn = cut_to_nearest

  def forward(self, X: torch.Tensor) -> torch.Tensor:
    item = X.cpu().numpy()
    
    wh, wl = pywt.dwt(item[0], self.wavelet)
    out = torch.stack((torch.from_numpy(wh), torch.from_numpy(wl)))
    
    if self.ctn is not None:
      out = out[:,:-1 * (out.shape[-1] % self.ctn)]
    
    return out

In [31]:
class AudioAugmentations(nn.Module):
  def __init__(
      self, 
      stretch_rate=0.8,
      fixed_rate=True,
      freq_mask_param=30,
      time_mask_param=80
  ):
    super(AudioAugmentations, self).__init__()
    self.aug = nn.Sequential(
        torchaudio.transforms.TimeStretch(stretch_rate, fixed_rate=fixed_rate),
        torchaudio.transforms.FrequencyMasking(freq_mask_param=freq_mask_param),
        torchaudio.transforms.TimeMasking(time_mask_param=time_mask_param),
    )
  
  def forward(self, X: torch.Tensor) -> torch.Tensor:
    return self.aug(X)

In [32]:
## REUSE IN INFERENCE NOTEBOOK

def resample(audio, current_sample_rate, desired_sample_rate=SAMPLE_RATE):
    resampler = torchaudio.transforms.Resample(orig_freq=current_sample_rate, new_freq=desired_sample_rate)
    resampled_audio = resampler(audio)
    return resampled_audio

def load_audio(audio_path, sample_rate=SAMPLE_RATE):
    audio, sr = torchaudio.load(audio_path)
    if sr != sample_rate:
        audio = resample(audio, sr, sample_rate)
    return audio

# Using librosa defaults for n_fft and hop_length
def get_melspec_transform(sample_rate=SAMPLE_RATE, n_fft=2048, hop_length=512, n_mels=128):
    return torchaudio.transforms.MelSpectrogram(
        sample_rate=sample_rate,
        n_fft=n_fft,
        hop_length=hop_length,
        n_mels=n_mels,
    )

# Using librosa defaults for top_db
def get_melspec_db_transform(stype='power', top_db=80):
    return torchaudio.transforms.AmplitudeToDB(
        stype=stype,
        top_db=top_db
    )

# Copied from torchaudio/transforms/_transforms.py (to avoid converting to melspec twice)
dct_mat = F_audio.create_dct(40, 128, "ortho")
def get_mfcc_from_melspec(melspec):
    return torch.matmul(melspec.transpose(-1, -2), dct_mat).transpose(-1, -2)

def normalize_tensor(tensor):
    min_val = torch.min(tensor)
    max_val = torch.max(tensor)
    if max_val - min_val == 0:
        return tensor
    else:
        return (tensor - min_val) / (max_val - min_val)

In [33]:
filepaths = glob.glob(f"{DATA_PATH}/test_soundscapes/*.ogg")
print(f"filepaths length: {len(filepaths)} (amount of test soundscapes)")

filepaths length: 1 (amount of test soundscapes)


# Inference

In [34]:
from torchvision import transforms

debug = False
simulate_200_files = False

if simulate_200_files:
    filepaths = [filepaths[0] for i in range(200)] # simulate submission
    print(f"filepaths length: {len(filepaths)} after simulation additions")

label_encoder = get_label_encoder()
model = get_model(out_features=len(label_encoder.classes_), device=DEVICE, pretrained=False, load_state_dict=True)
model.eval()

MIN_WINDOW = AUDIO_LENGTH_S * SAMPLE_RATE
melspec_transform = get_melspec_transform(n_mels=128)
melspec_db_transform = get_melspec_db_transform()
wave_transform = WaveletTransformSingle(pywt.Wavelet('sym4'))
resize = transforms.Resize((128, 313), antialias=True)

def infer(filepath):
    all_predictions = []
    name = Path(filepath).stem
    audio = load_audio(filepath)
    audio_len_s = audio.shape[1] / SAMPLE_RATE
    debug and print(f"Infering file {filepath} with length {audio_len_s} s")
    n_crops = int(audio_len_s // 5)
    for i in range(n_crops):
        debug and print(f"Crop {i} / {n_crops}")
        debug and print(f"Audio length: {len(audio)}")
        crop = audio[:, i*MIN_WINDOW:(i+1)*MIN_WINDOW]
        debug and print(f"Crop dimensions: {crop.shape}")
        melspec = melspec_db_transform(melspec_transform(crop))
        norm_melspec = normalize_tensor(melspec)
        melspec_wave = wave_transform(crop)
        wh, wl = melspec_wave[0], melspec_wave[1]
        wh_mel = melspec_db_transform(melspec_transform(wh))
        wl_mel = melspec_db_transform(melspec_transform(wl))
        norm_wh = normalize_tensor(resize(wh_mel.unsqueeze(0)))
        norm_wl = normalize_tensor(resize(wl_mel.unsqueeze(0)))
        mfcc = get_mfcc_from_melspec(melspec)
        norm_mfcc = normalize_tensor(resize(mfcc))
        features = torch.cat((norm_melspec, norm_wh, norm_wl, norm_mfcc), dim=0)
        debug and print(f"features shape: {features.shape}") # [4, 128, 313]
        features = features.unsqueeze(0) # add batch dimension (1)
        debug and print(f"features unsqueezed shape: {features.shape}") # [1, 4, 128, 313]
        with torch.no_grad():
            logit, proba = model(features)
        t = (i + 1) * 5
        all_predictions.append({"row_id": f'{name}_{t}',"predictions": proba})
        debug and print('---')
    return all_predictions

if debug:
    all_preds = []
    for filepath in tqdm(filepaths, desc='Infering files'):
        all_preds.append(infer(filepath))
else:
    parallel_task = (delayed(infer)(filepath) for filepath in tqdm(filepaths, desc='Infering files'))
    all_preds = Parallel(n_jobs=os.cpu_count())(parallel_task)

all_preds_flat = [item for sublist in all_preds for item in sublist]

print(f"all_preds length: {len(all_preds)}, all_preds_flat length: {len(all_preds_flat)}")

Loaded label encoder from ./label_encoder.joblib
Loaded model BirdMelspecClf with 5855040 parameters, pretained=False
No model starting with 5s_regnetY800MF_ found in ./


Infering files: 100%|██████████| 1/1 [00:00<00:00, 718.45it/s]


all_preds length: 1, all_preds_flat length: 120


In [35]:
all_preds_flat[100]['predictions'][0]

tensor([0.0039, 0.0038, 0.0036, 0.0037, 0.0039, 0.0038, 0.0037, 0.0036, 0.0037,
        0.0039, 0.0038, 0.0038, 0.0038, 0.0037, 0.0039, 0.0036, 0.0040, 0.0037,
        0.0038, 0.0039, 0.0040, 0.0038, 0.0038, 0.0037, 0.0039, 0.0037, 0.0040,
        0.0037, 0.0038, 0.0037, 0.0036, 0.0038, 0.0036, 0.0038, 0.0038, 0.0037,
        0.0039, 0.0038, 0.0036, 0.0036, 0.0040, 0.0036, 0.0039, 0.0040, 0.0038,
        0.0037, 0.0036, 0.0039, 0.0037, 0.0038, 0.0039, 0.0038, 0.0038, 0.0038,
        0.0039, 0.0038, 0.0039, 0.0038, 0.0038, 0.0039, 0.0039, 0.0039, 0.0040,
        0.0040, 0.0037, 0.0036, 0.0038, 0.0040, 0.0038, 0.0036, 0.0038, 0.0038,
        0.0036, 0.0037, 0.0037, 0.0040, 0.0036, 0.0039, 0.0039, 0.0036, 0.0037,
        0.0036, 0.0037, 0.0038, 0.0036, 0.0039, 0.0039, 0.0036, 0.0038, 0.0037,
        0.0037, 0.0037, 0.0037, 0.0034, 0.0037, 0.0037, 0.0036, 0.0036, 0.0037,
        0.0039, 0.0038, 0.0037, 0.0038, 0.0036, 0.0036, 0.0038, 0.0039, 0.0038,
        0.0038, 0.0037, 0.0039, 0.0036, 

In [36]:
df = pd.concat([
    pd.DataFrame({'row_id': [p['row_id'] for p in all_preds_flat]}), 
    pd.DataFrame(torch.stack([p['predictions'][0] for p in all_preds_flat]).numpy(), columns=label_encoder.classes_)
], axis=1)

df

,row_id,abethr1,abhori1,abythr1,afbfly1,afdfly1,afecuc1,affeag1,afgfly1,afghor1,...,yebsto1,yeccan1,yefcan,yelbis1,yenspu1,yertin1,yesbar1,yespet1,yetgre1,yewgre1
0,soundscape_29201_5,0.003870,0.003786,0.003631,0.003702,0.003928,0.003805,0.003747,0.003640,0.003671,...,0.003826,0.003856,0.003782,0.003615,0.003755,0.003804,0.003850,0.003718,0.003778,0.003703
1,soundscape_29201_10,0.003872,0.003786,0.003628,0.003698,0.003926,0.003801,0.003751,0.003645,0.003666,...,0.003816,0.003860,0.003780,0.003610,0.003756,0.003807,0.003849,0.003715,0.003780,0.003699
2,soundscape_29201_15,0.003869,0.003788,0.003635,0.003703,0.003929,0.003804,0.003748,0.003639,0.003670,...,0.003821,0.003857,0.003786,0.003616,0.003750,0.003800,0.003846,0.003724,0.003778,0.003708
3,soundscape_29201_20,0.003868,0.003786,0.003632,0.003703,0.003929,0.003805,0.003747,0.003641,0.003669,...,0.003820,0.003857,0.003785,0.003616,0.003751,0.003804,0.003849,0.003721,0.003778,0.003706
4,soundscape_29201_25,0.003871,0.003786,0.003632,0.003703,0.003931,0.003805,0.003746,0.003639,0.003671,...,0.003824,0.003855,0.003787,0.003616,0.003751,0.003806,0.003852,0.003723,0.003781,0.003705
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,soundscape_29201_580,0.003873,0.003794,0.003635,0.003699,0.003932,0.003806,0.003742,0.003636,0.003668,...,0.003825,0.003856,0.003789,0.003615,0.003745,0.003805,0.003849,0.003725,0.003783,0.003704
116,soundscape_29201_585,0.003874,0.003792,0.003633,0.003699,0.003935,0.003804,0.003744,0.003636,0.003666,...,0.003825,0.003859,0.003789,0.003612,0.003748,0.003804,0.003852,0.003722,0.003781,0.003703
117,soundscape_29201_590,0.003875,0.003791,0.003634,0.003699,0.003934,0.003803,0.003744,0.003638,0.003665,...,0.003824,0.003859,0.003786,0.003611,0.003746,0.003806,0.003851,0.003721,0.003783,0.003703
118,soundscape_29201_595,0.003876,0.003792,0.003637,0.003700,0.003932,0.003802,0.003745,0.003635,0.003659,...,0.003819,0.003862,0.003790,0.003612,0.003743,0.003802,0.003846,0.003727,0.003781,0.003705


In [37]:
df.to_csv('submission.csv', index=False)

In [38]:
!ls submission.csv

submission.csv
